In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
import io
Income_train_Set = pd.read_csv('tcd-ml-1920-group-income-train.csv') # index_col = 0把instance当作索引，否则会另外建立索引
Income_Pred_Set = pd.read_csv('tcd-ml-1920-group-income-test.csv')

# New Section

In [0]:
Income_train_Set.head()

In [0]:
_new_column = {'Year of Record':'Year',
               'Housing Situation':'Housing',
               'Crime Level in the City of Employement':'Crime',
               'Work Experience in Current Job [years]':'Experience',
               'Satisfation with employer':'Satisfaction',
               'Size of City':'City',
               'University Degree':'Degree',
               'Wears Glasses':'Glasses',
               'Hair Color':'Hair',
               'Body Height [cm]':'Height',
               'Yearly Income in addition to Salary (e.g. Rental Income)':'Add_Income',
               'Total Yearly Income [EUR]': 'Income'}
Income_train_Set = Income_train_Set.rename(_new_column, axis = 1)
Income_Pred_Set = Income_Pred_Set.rename(_new_column, axis = 1)

In [0]:
Income_train_Set.head()
Income_Pred_Set.head()

In [0]:
type(Income_train_Set['Add_Income'])

In [0]:
#remove EUR in Income column
Income_train_Set['Add_Income'] = Income_train_Set['Add_Income'].map(lambda x : x.replace('EUR', ''))
Income_Pred_Set['Add_Income'] = Income_Pred_Set['Add_Income'].map(lambda x : x.replace('EUR', ''))

In [0]:
#convert Object to float
Income_train_Set['Add_Income'] = pd.to_numeric(Income_train_Set['Add_Income'])
Income_Pred_Set['Add_Income'] = pd.to_numeric(Income_Pred_Set['Add_Income'])

In [0]:
Income_Pred_Set.info()

In [0]:
#test.plot.scatter(x = 'Instance', y = 'Income')

In [0]:
#Rand_Set_1 = Income_train_Set.sample(frac = .1) randomly select rows

In [0]:
Income_train_Set.info()

#### IQR

In [0]:
def subset_by_iqr(df, column, whisker_width=1.1):
    """Remove outliers from a dataframe by column, including optional 
       whiskers, removing rows for which the column value are 
       less than Q1-1.5IQR or greater than Q3+1.5IQR.
    Args:
        df (`:obj:pd.DataFrame`): A pandas dataframe to subset
        column (str): Name of the column to calculate the subset from.
        whisker_width (float): Optional, loosen the IQR filter by a
                               factor of `whisker_width` * IQR.
    Returns:
        (`:obj:pd.DataFrame`): Filtered dataframe
    """
    # Calculate Q1, Q2 and IQR
    q1 = df[column].quantile(0.25)                 
    q3 = df[column].quantile(0.75)
    iqr = q3 - q1
    # Apply filter with respect to IQR, including optional whiskers
    filter = (df[column] >= q1 - whisker_width*iqr) & (df[column] <= q3 + whisker_width*iqr)
    return df.loc[filter]                  

In [0]:
Income_train_Set = subset_by_iqr(Income_train_Set, 'Income', whisker_width=4.0)

In [0]:
Income_train_Set.plot.scatter(x = 'Instance', y = 'Income')

#### Important for de-concat----------------

In [0]:
 Income_train_Set.shape ## Important for de-concat

#### ---------------------------------

In [0]:
Concat_Set = pd.concat([Income_train_Set,Income_Pred_Set],ignore_index=True)

In [0]:
from sklearn.impute import SimpleImputer
# fill nan strategy: most_frequent value
'''
Year = Income_train_Set['Hair'].value_counts()   #median();mean();value_counts
Year
most_frequent:
    *Gender:male
    *Degree:Bachelor
    *Hair:Black
    *country:Honduras
'''

In [0]:
dt = Concat_Set.dtypes


In [0]:
fillna_mode = SimpleImputer(strategy = "most_frequent" ) #fill nan with most frequent

In [0]:
Year = Concat_Set['Year'].values.reshape(-1,1)
Concat_Set['Year'] = fillna_mode.fit_transform(Year)

In [0]:
fillna_dict = {'Satisfaction':'Average',
               'Gender':'male',
               'Country': 'Honduras',
               'Profession': 'payment analyst',
               'Degree': 'Bachelor',
               'Hair': 'Black'
               }

In [0]:
for key in fillna_dict.keys():
    Concat_Set[key] = Concat_Set[key].fillna(fillna_dict[key])
# using sklearn module get longer runtime

In [0]:
Concat_Set.isnull().sum()

In [0]:
def create_cat_con(df,cats,cons,normalize=True):
    for i,cat in enumerate(cats):
        vc = df[cat].value_counts(dropna=False, normalize=normalize).to_dict() #normalize = true 显示不同value占比
        nm = cat + '_FE_FULL' 
        df[nm] = df[cat].map(vc) 
        df[nm] = df[nm].astype('float32')
        for j,con in enumerate(cons):
            new_col = cat +'_'+ con
            print('timeblock frequency encoding:', new_col)
            df[new_col] = df[cat].astype(str)+'_'+df[con].astype(str)  
            temp_df = df[new_col]
            fq_encode = temp_df.value_counts(normalize=True).to_dict()
            df[new_col] = df[new_col].map(fq_encode)
            df[new_col] = df[new_col]/df[cat+'_FE_FULL']
    return df

In [0]:
cats = ['Year', 'Housing', 'Experience',
        'Satisfaction', 'Gender', 'Age',
        'Country', 'Profession', 'Degree', 'Glasses', 'Hair']
cons = ['Crime', 'City', 'Height', 'Add_Income']

data = create_cat_con(Concat_Set,cats,cons)
'done'

In [0]:
for col in Income_train_Set.dtypes[Income_train_Set.dtypes == 'object'].index.tolist():
    feat_le = LabelEncoder()
    feat_le.fit(Concat_Set[col].unique().astype(str))
    Concat_Set[col] = feat_le.transform(data[col].astype(str))
    
del_col = set(['Income','Instance'])
features_col =  list(set(Concat_Set) - del_col)



In [0]:
X_train,X_test = Concat_Set[features_col].iloc[:1025718],data[features_col].iloc[1025719:]
Y_train = data['Income'].iloc[:1025718]
X_test_id = data['Instance'].iloc[1025719:]
x_train,x_val,y_train,y_val = train_test_split(X_train,Y_train,test_size=0.2,random_state=1234)

In [0]:
parameters = {
              'max_depth': [15, 20, 25, 30, 35],
              'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.15],
              'feature_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_fraction': [0.6, 0.7, 0.8, 0.9, 0.95],
              'bagging_freq': [2, 4, 5, 6, 8],
              'lambda_l1': [0, 0.1, 0.4, 0.5, 0.6],
              'lambda_l2': [0, 10, 15, 35, 40],
              'cat_smooth': [1, 10, 15, 20, 35]}
params = {
          'max_depth': 35,
          'learning_rate': 0.02,
          "boosting": "gbdt",
          "bagging_seed": 10000,
          "metric": 'mae',
          "verbosity": -1,
          "subsample" : 0.3,
          "lambda_l1" : 0.6,
          "lambda_l2" : 10,
          "cat_smooth": 1
         }
trn_data = lgb.Dataset(x_train, label=y_train)
val_data = lgb.Dataset(x_val, label=y_val)
# test_data = lgb.Dataset(X_test)
clf = lgb.train(params, trn_data, 100000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
pre_test_lgb = clf.predict(X_test)
'done'


sub_df = pd.DataFrame({'Instance':X_test_id,
                       'Income':pre_test_lgb})
sub_df.to_csv("predictions.csv",index=False)
files.download('predictions.csv') 

In [0]:
from sklearn.metrics import mean_squared_error
pre_val_lgb = clf.predict(x_val)
val_mse = mean_squared_error(y_val,pre_val_lgb)
val_rmse = np.sqrt(val_mse)
val_rmse

In [0]:
#Raw_Data.plot.scatter(x = 'Instance', y = 'Income') == 
#JW: salary_data = Income_train_Set[Income_train_Set['Income'] < 400000]

In [0]:
sub_df = pd.DataFrame({'Instance':X_test_id,
                       'Income':pre_test_lgb})
sub_df.head()

In [0]:
sub_df.to_csv("sub191015_6.csv",index=False)
'done'

In [0]:
import io
files.download('sub191015_6.csv') 